<a href="https://colab.research.google.com/github/itimes-digital/mineracao-texto/blob/main/analise_de_texto_para_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import nltk
import pandas as pd
import re
from unicodedata import normalize

In [107]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [108]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### Preparação dos dados

In [109]:
base = pd.read_csv('sample_data/lista_classificada.csv', sep=';', engine='python')

In [110]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",Nao recebi email para validar o cadastro NÃO C...
1,"Cadastro, Login, Senha",Não consigo concluir o cadastro.
2,"Loja C&R, Estoque",Nunca realizei compras pelo app. Pois a forma ...
3,Entrega,"Muito demora pra entregar.Farmácia é urgência,..."
4,Entrega,"Péssimo no que se refere ao item ENTREGA, pois..."


In [111]:
base['Comentario'] = base['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
base['Comentario'] = base['Comentario'].map(lambda x: normalize('NFKD', x.lower()).encode('ASCII','ignore').decode('ASCII'))

In [112]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",nao recebi email para validar o cadastro nao c...
1,"Cadastro, Login, Senha",nao consigo concluir o cadastro
2,"Loja C&R, Estoque",nunca realizei compras pelo app pois a forma d...
3,Entrega,muito demora pra entregarfarmacia e urgencia q...
4,Entrega,pessimo no que se refere ao item entrega pois ...


In [113]:
base['Categoria'].value_counts()

Lentidão, Trava, Crash, Performance    121
Entrega                                 85
Pedido cancelamento                     39
Erro finalização                        38
Erro PIX                                34
Loja C&R, Estoque                       27
Preço, Prazo, Promoção, Stix            21
Erro cartão de crédito                  11
Cesta                                   10
Busca                                    7
Novas Funcionalidades                    6
Cadastro, Login, Senha                   5
Problema não detalhado                   3
Name: Categoria, dtype: int64

In [114]:
base = base.sample(frac=1)

In [115]:
base_treinamento = base.iloc[:350,:]
base_teste = base.iloc[350:,:]

In [116]:
base_treinamento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 191 to 50
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   350 non-null    object
 1   Comentario  350 non-null    object
dtypes: object(2)
memory usage: 8.2+ KB


In [117]:
base_teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 314 to 49
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   57 non-null     object
 1   Comentario  57 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [118]:
base_treinamento = base_treinamento.reset_index(drop=True)
base_teste = base_teste.reset_index(drop=True)

In [119]:
base_teste.head()

,Categoria,Comentario
0,Pedido cancelamento,nao da pra cancelar o pedido
1,"Cadastro, Login, Senha",nao recebi email para validar o cadastro nao c...
2,Entrega,forma de entrega(retirar em loja) deveria vir ...
3,Entrega,muita demora pra entregar
4,"Lentidão, Trava, Crash, Performance",esta travando


In [120]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [121]:
def removeStopWordsAplicandoStemmer(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords]
        frases.append((sem_stop, base.loc[i,"Categoria"]))
    return frases;

In [122]:
frasesComStemmingTreinamento = removeStopWordsAplicandoStemmer(base_treinamento);
frasesComStemmingTeste = removeStopWordsAplicandoStemmer(base_teste);
#print(frasesComStemming)

In [123]:
def buscaPalavras(frases):
    todasPalavras = [];
    for (palavras, categoria) in frases:
        todasPalavras.extend(palavras);
    return todasPalavras;

In [124]:
palavrasTreinamento = buscaPalavras(frasesComStemmingTreinamento);
palavrasTeste = buscaPalavras(frasesComStemmingTeste);
#print(palavras)

In [125]:
def buscaFrequencia(palavras):
    palavras = nltk.FreqDist(palavras);
    return palavras;

In [126]:
frequenciaTreinamento = buscaFrequencia(palavrasTreinamento);
frequenciaTeste = buscaFrequencia(palavrasTeste);
#print(frequencia.most_common(50))

In [127]:
def buscaPalavrasUnicas(frequencia):
    freq = frequencia.keys();
    return freq;

In [128]:
palavrasUnicasTreinamento = buscaPalavrasUnicas(frequenciaTreinamento);
palavrasUnicasTeste = buscaPalavrasUnicas(frequenciaTeste);
#print(palavrasUnicas)

In [129]:
def extratorPalavras(documento):
    doc = set(documento);
    caracteristicas = {};
    for palavras in palavrasUnicasTreinamento:
        caracteristicas['%s' % palavras] = (palavras in doc);
    return caracteristicas;

In [130]:
caracteristicasFrases = extratorPalavras({'am', 'nov', 'dia'});
#print(caracteristicasFrases)

### Realizando a classificação

In [131]:
baseCompletaTreinamento = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTreinamento);
baseCompletaTeste = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTeste);
#print(baseCompleta);

In [132]:
classificador = nltk.NaiveBayesClassifier.train(baseCompletaTreinamento);

In [133]:
print(classificador.labels())

['Lentidão, Trava, Crash, Performance', 'Pedido cancelamento', 'Entrega', 'Erro PIX', 'Erro finalização', 'Preço, Prazo, Promoção, Stix', 'Loja C&R, Estoque', 'Problema não detalhado', 'Busca', 'Novas Funcionalidades', 'Erro cartão de crédito', 'Cesta', 'Cadastro, Login, Senha']


In [134]:
 #usando a mesma amostra do modelo treinado
print(nltk.classify.accuracy(classificador, baseCompletaTreinamento))

0.6914285714285714


In [135]:
#usando a amostra de teste
print(nltk.classify.accuracy(classificador, baseCompletaTeste)) 

0.5087719298245614


In [136]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                  cancel = True           Pedido : Lentid =     67.6 : 1.0
                     pix = True           Erro P : Lentid =     38.7 : 1.0
                   carta = True           Erro c : Entreg =     38.5 : 1.0
                 cadastr = True           Cadast : Entreg =     34.1 : 1.0
                     peg = True           Proble : Lentid =     26.5 : 1.0
                  confus = True           Proble : Lentid =     26.5 : 1.0
                   demor = True           Proble : Lentid =     26.5 : 1.0
                  adiant = True           Proble : Lentid =     26.5 : 1.0
                  entreg = True           Entreg : Lentid =     26.3 : 1.0
                    salv = True           Novas  : Lentid =     25.2 : 1.0
None


In [138]:
erros = []

for (frase, classe) in baseCompletaTeste:
    previsao = classificador.classify(frase)

    if previsao != classe:
        erros.append((classe, previsao, frase))

for (flasse, previsao, frase) in erros:
    # Classe real | previsão da classe | frase
    print(classe, previsao, frase)

Entrega Lentidão, Trava, Crash, Performance {'vez': False, 'lent': False, 'dem': False, 'confirm': False, 'compr': False, 'terc': False, 'tent': False, 'send': False, '2': False, 'cancel': True, 'drogara': False, 'qualqu': False, 'justifoc': False, 'produt': False, 'exist': False, 'disponi': False, 'entreg': False, 'promet': False, 'nao': True, 'cumpr': False, 'precid': False, 'ds': False, '4h': False, 'prim': False, 'pedidofo': False, 'feit': False, '11h': False, 'sao': False, '16:36': False, 'nad': False, 'sab': False, 'porqu': False, 'part': False, 'tod': False, 'hor': False, 'ped': True, 'atual': False, 'pouc': False, 'cust': False, 'acess': False, 'app': False, 'tr': False, 'aind': False, 'princip': False, 'ativ': False, 'ofert': False, 'sai': False, 'aplic': False, 'merd': False, 'pq': False, 'vc': False, '90': False, 'ano': False, 'liter': False, 'cheg': False, 'remedi': False, 'pra': True, 'pressa': False, 'ja': False, 'tav': False, 'mort': False, 'dia': False, '02/05': False, 

In [139]:
from nltk.metrics import ConfusionMatrix

esperado = []
previsto = []

for (frase, classe) in baseCompletaTeste:
    previsao = classificador.classify(frase);
    previsto.append(previsao);
    esperado.append(classe);

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

                                    |                 L          |
                                    |                 e          |
                                    |                 n          |
                                    |                 t          |
                                    |                 i          |
                                    |                 d          |
                                    |                 ã          |
                                    |                 o        P |
                                    |                 ,        r |
                                    |                          e |
                                    |                 T        ç |
                                    |                 r        o |
                                    |                 a        , |
                                    |  C              v          |
                                    |  a              a       

### Montagem de uma frase para testar classificação.

In [140]:
testeFrase = 'Meu pedido não fecha adoentado'

In [141]:
def aplicandoStemmerFrase(frase):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for (palavrasTreinamento) in frase.split():
        com_stem_sem_stop = [p for p in palavrasTreinamento.split()];
        frases.append(str(stemmer.stem(com_stem_sem_stop[0])));
    return frases;

In [142]:
fraseComStemming = aplicandoStemmerFrase(testeFrase)
print(fraseComStemming)

['meu', 'ped', 'não', 'fech', 'adoent']


In [143]:
novaFrase = extratorPalavras(fraseComStemming);
#print(novaFrase)

In [144]:
print(classificador.classify(novaFrase))

Lentidão, Trava, Crash, Performance


In [145]:
distribuicao = classificador.prob_classify(novaFrase)
for classe in distribuicao.samples():
    print("%s: %f" % (classe, distribuicao.prob(classe)))

Lentidão, Trava, Crash, Performance: 0.999818
Pedido cancelamento: 0.000000
Entrega: 0.000182
Erro PIX: 0.000000
Erro finalização: 0.000000
Preço, Prazo, Promoção, Stix: 0.000000
Loja C&R, Estoque: 0.000000
Problema não detalhado: 0.000000
Busca: 0.000000
Novas Funcionalidades: 0.000000
Erro cartão de crédito: 0.000000
Cesta: 0.000000
Cadastro, Login, Senha: 0.000000


### Montagem dos dados novos para classificação

In [146]:
baseSemCategoria = pd.read_csv('sample_data/lista.csv', sep=';', engine='python')

In [147]:
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: normalize('NFKD', x.lower()).encode('ASCII','ignore').decode('ASCII'))

In [148]:
def removeStopWordsAplicandoStemmerSemCategoria(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords];
        frases.append(sem_stop);
    return frases;

In [149]:
frasesComStemmingSemCategoria = removeStopWordsAplicandoStemmerSemCategoria(baseSemCategoria)
#print(frasesComStemmingSemCategoria)

In [150]:
baseCompletaSemClassificacao = nltk.classify.apply_features(extratorPalavras, frasesComStemmingSemCategoria);

In [151]:
#print(baseCompletaSemClassificacao)